In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dropout, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shiva\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shiva\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [5]:
file_path = "C:/Users/shiva/Desktop/Guvi/FakeNews_Det_Capstone/WELFake_Dataset.csv"
data = pd.read_csv(file_path)
#data.head()


In [6]:
data.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [7]:

!pip install swifter

     ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
      --------------------------------------- 0.0/1.2 MB 217.9 kB/s eta 0:00:06
     - -------------------------------------- 0.0/1.2 MB 393.8 kB/s eta 0:00:03
     -- ------------------------------------- 0.1/1.2 MB 409.6 kB/s eta 0:00:03
     --- ------------------------------------ 0.1/1.2 MB 438.1 kB/s eta 0:00:03
     --- ------------------------------------ 0.1/1.2 MB 504.4 kB/s eta 0:00:03
     ---- ----------------------------------- 0.1/1.2 MB 481.4 kB/s eta 0:00:03
     ----- ---------------------------------- 0.2/1.2 MB 492.3 kB/s eta 0:00:03
     ----- ---------------------------------- 0.2/1.2 MB 477.7 kB/s eta 0:00:03
     ------ --------------------------------- 0.2/1.2 MB 479.2 kB/s eta 0:00:03
     ------- -------------------------------- 0.2/1.2 MB 492.1 kB/s eta 0:00:02
     -------- ------------------------------- 0.2/1.2 MB 503.2 kB/s eta 0:00:02
     --------- ------------------------------ 0.3


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    try:
        text = re.sub(r'[^a-zA-Z]', ' ', text)
        tokens = word_tokenize(text)
        tokens = [word.lower() for word in tokens]
        tokens = [word for word in tokens if word not in stop_words]
        return ' '.join(tokens)
    except TypeError:
        return ''


In [9]:
import swifter
data['clean_text'] = data['text'].swifter.apply(preprocess_text)
data['clean_title_text'] = data['title'] + " " + data['clean_text']

     

c:\Users\shiva\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Pandas Apply: 100%|██████████| 72134/72134 [02:45<00:00, 436.99it/s]


In [11]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
nltk.download('stopwords')
nltk.download('punkt')
file_path = "C:/Users/shiva/Desktop/Guvi/FakeNews_Det_Capstone/WELFake_Dataset.csv"
data = pd.read_csv(file_path)
data.head()
data['text'] = data['text'].astype(str)
data['title'] = data['title'].astype(str)
data.fillna('', inplace=True)
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    if not isinstance(text, str):
        text = str(text)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens]
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)
data['clean_text'] = data['text'].apply(preprocess_text)
data['clean_title'] = data['title'].apply(preprocess_text)
data['clean_title_text'] = data['clean_title'] + " " + data['clean_text']

     

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shiva\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shiva\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(data['clean_title_text'])
sequences = tokenizer.texts_to_sequences(data['clean_title_text'])
padded_sequences = pad_sequences(sequences, maxlen=500)

X = padded_sequences
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=500))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))

c:\Users\shiva\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
902/902 ━━━━━━━━━━━━━━━━━━━━ 687s 746ms/step - accuracy: 0.8642 - loss: 0.3165 - val_accuracy: 0.9306 - val_loss: 0.1795
Epoch 2/5
902/902 ━━━━━━━━━━━━━━━━━━━━ 684s 759ms/step - accuracy: 0.9410 - loss: 0.1635 - val_accuracy: 0.9496 - val_loss: 0.1558
Epoch 3/5
902/902 ━━━━━━━━━━━━━━━━━━━━ 679s 753ms/step - accuracy: 0.9534 - loss: 0.1357 - val_accuracy: 0.8452 - val_loss: 0.2665
Epoch 4/5
902/902 ━━━━━━━━━━━━━━━━━━━━ 709s 787ms/step - accuracy: 0.9558 - loss: 0.1238 - val_accuracy: 0.9574 - val_loss: 0.1209
Epoch 5/5
902/902 ━━━━━━━━━━━━━━━━━━━━ 717s 795ms/step - accuracy: 0.9749 - loss: 0.0766 - val_accuracy: 0.9713 - val_loss: 0.0845


In [14]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))
print("F1-Score:", f1_score(y_test, y_pred))

model.save('fake_news_classifier.h5')

0

451/451 ━━━━━━━━━━━━━━━━━━━━ 52s 113ms/step


              precision    recall  f1-score   support

           0       0.97      0.97      0.97      7089
           1       0.97      0.97      0.97      7338

    accuracy                           0.97     14427
   macro avg       0.97      0.97      0.97     14427
weighted avg       0.97      0.97      0.97     14427

F1-Score: 0.9717983651226159


0

In [15]:

!pip install flask_ngrok


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/101.7 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/101.7 kB ? eta -:--:--
   --------------- ----------------------- 41.0/101.7 kB 487.6 kB/s eta 0:00:01
   --------------- ----------------------- 41.0/101.7 kB 487.6 kB/s eta 0:00:01
   ----------------------- --------------- 61.4/101.7 kB 409.6 kB/s eta 0:00:01
   ----------------------- --------------- 61.4/101.7 kB 409.6 kB/s eta 0:00:01
   ----------------------------------- --- 92.2/101.7 kB 350.1 kB/s eta 0:00:01
   -------------------------------------- 101.7/101.7 kB 344.0 kB/s eta 0:00:00


In [16]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

model_path = 'fake_news_classifier.h5'
model = tf.keras.models.load_model(model_path)
tokenizer = Tokenizer(num_words=5000)
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens]
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

def predict(news_text):
    clean_news = preprocess_text(news_text)
    seq = tokenizer.texts_to_sequences([clean_news])
    padded = pad_sequences(seq, maxlen=500)
    prediction = (model.predict(padded) > 0.5).astype("int32")[0][0]
    return 'real' if prediction == 1 else 'fake'

if __name__ == '__main__':
    test_articles = [
        "This is a piece of fake news about a celebrity scandal.",
        "The stock market reached an all-time high today due to strong economic growth.",
        "Unbelievable! Aliens have landed on Earth and are communicating with humans."
    ]

    for article in test_articles:
        result = predict(article)
        print(f"Article: {article}\nPrediction: {result}\n")

     

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shiva\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shiva\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 956ms/step
Article: This is a piece of fake news about a celebrity scandal.
Prediction: real

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Article: The stock market reached an all-time high today due to strong economic growth.
Prediction: real

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
Article: Unbelievable! Aliens have landed on Earth and are communicating with humans.
Prediction: real

